In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import NegativeClassOptimization.config as config
#from scipy.stats import entropy
import torch
import torch.nn as nn
from NegativeClassOptimization.preprocessing import *



In [2]:
df_ex = pd.read_csv('../data/Slide_importance/1ADQ_A_Task4_Slice_ParaEpi.txt', sep='\t')

In [3]:
df_ex

,Slide,Label,hotspot_ID,seqAGEpitope,motifAGEpitope,agregatesAGEpitope,chemicalAGEpitope,seqABParatope,motifABParatope,agregatesABParatope,chemicalABParatope,AAcompoFullSlice,sizeCDR3
0,LLGLYWYFDVW,1ADQ_A,1ADQ_A_H1,S1W1V1D1V2Q1V3N1A1K1T2V1L2H1N1Y1,X1--X1--X1X1--X2X1X3--X1X1X1--X2X1X2X1--X1--X1,S1--W1--V1D1--V2Q1V3--N1A1K1--T2V1L2H1--N1--Y1,p1--r1--n1c1--n2p1n3--p1n1c1--p2n1n2c1--p1--r1,L3L4G3L1Y2Y2F2V2W2*,X3X4X3X1X2--X2X2--X2X2*,L3L4G3L1Y2--Y2F2--V2W2*,n3n4n3n1r2--r2r2--n2r2*,0_0_0.0909091_0_0.0909091_0.0909091_0_0_0_0.27...,15
1,YYSNYWYFDVW,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,Y2Y3N2Y1W3Y1F4D1V4W3*,X2X3--X2X1X3X1X4X1X4X3*,Y2Y3--N2Y1W3Y1F4D1V4W3*,r2r3--p2r1r3r1r4c1n4r3*,0_0_0.0909091_0_0.0909091_0_0_0_0_0_0_0.090909...,14
2,AILLLRGKAYW,1ADQ_A,1ADQ_A_H1,L1R1P1W1V1D2V1Q1V1A1K1T1V1L1H1Q3N1W1L1Y1,X1--X1--X1--X1--X1X2--X1X1X1--X1X1--X1X1X1X1X3...,L1--R1--P1--W1--V1D2--V1Q1V1--A1K1--T1V1L1H1Q3...,n1--c1--n1--r1--n1c2--n1p1n1--n1c1--p1n1n1c1p3...,A2I3L4L3L1R1G1K1A2Y1W4*,X2X3X4X3X1X1X1X1X2X1X4*,A2I3L4L3L1R1G1K1A2Y1W4*,n2n3n4n3n1c1n1c1n2r1r4*,0.181818_0_0_0_0_0.0909091_0_0.0909091_0.09090...,12
3,YYDYDWYFDVW,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,Y2Y3Y2D1W3Y1F4D1V4W3*,X2X3--X2X1X3X1X4X1X4X3*,Y2Y3--Y2D1W3Y1F4D1V4W3*,r2r3--r2c1r3r1r4c1n4r3*,0_0_0.272727_0_0.0909091_0_0_0_0_0_0_0_0_0_0_0...,15
4,SLIYYGYGAWF,1ADQ_A,1ADQ_A_H1,L1R1P1W1V1D2V1Q1V1A1K1T1V1L1H1Q3N1W1L1Y1,X1--X1--X1--X1--X1X2--X1X1X1--X1X1--X1X1X1X1X3...,L1--R1--P1--W1--V1D2--V1Q1V1--A1K1--T1V1L1H1Q3...,n1--c1--n1--r1--n1c2--n1p1n1--n1c1--p1n1n1c1p3...,S2L3I4Y3Y1G1Y1G1A2W1F4*,X2X3X4X3X1X1X1X1X2X1X4*,S2L3I4Y3Y1G1Y1G1A2W1F4*,p2n3n4r3r1n1r1n1n2r1r4*,0.0909091_0_0_0_0.0909091_0.181818_0_0.0909091...,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48802,CVRWRLGFAYW,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,C3V4R1W4R1L3G1F2Y3W2*,X3X4X1X4X1X3X1X2--X3X2*,C3V4R1W4R1L3G1F2--Y3W2*,n3n4c1r4c1n3n1r2--r3r2*,0.0909091_0.0909091_0_0_0.0909091_0.0909091_0_...,11
48803,CVSLWLLYYYA,1ADQ_A,1ADQ_A_H1,S1W1V1D1V2Q1V3N1A1K1T2V1L2H1N1Y1,X1--X1--X1X1--X2X1X3--X1X1X1--X2X1X2X1--X1--X1,S1--W1--V1D1--V2Q1V3--N1A1K1--T2V1L2H1--N1--Y1,p1--r1--n1c1--n2p1n3--p1n1c1--p2n1n2c1--p1--r1,C2V2L1W3L4L3Y2Y2A2*,X2X2--X1X3X4X3X2X2--X2*,C2V2--L1W3L4L3Y2Y2--A2*,n2n2--n1r3n4n3r2r2--n2*,0.0909091_0.0909091_0_0_0_0_0_0_0_0.272727_0_0...,15
48804,VSLWLLYYYAM,1ADQ_A,1ADQ_A_H1,S1W1V1D1V2Q1V3N1A1K1T2V1L2H1N1Y1,X1--X1--X1X1--X2X1X3--X1X1X1--X2X1X2X1--X1--X1,S1--W1--V1D1--V2Q1V3--N1A1K1--T2V1L2H1--N1--Y1,p1--r1--n1c1--n2p1n3--p1n1c1--p2n1n2c1--p1--r1,V2L2W2L3L4Y3Y1A2M2*,X2--X2X2X3X4X3X1--X2X2*,V2--L2W2L3L4Y3Y1--A2M2*,n2--n2r2n3n4r3r1--n2n2*,0.0909091_0_0_0_0_0_0_0_0_0.272727_0.0909091_0...,15
48805,AWRERLPLVFV,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,A2W3E2R1L3P1L4V1F4V3*,X2X3--X2X1X3X1X4X1X4X3*,A2W3--E2R1L3P1L4V1F4V3*,n2r3--c2c1n3n1n4n1r4n3*,0.0909091_0_0_0.0909091_0.0909091_0_0_0_0_0.18...,13


In [24]:
df.sizeCDR3.median()

15.0

In [11]:
ag_stats = dict()
for ag in config.ANTIGENS:
    ag_path = glob.glob(f'../data/Slide_importance/{ag}_*_Task4_Slice_ParaEpi.txt')[0]
    df = pd.read_csv(ag_path, sep='\t')
    hot_spots = len(df.hotspot_ID.unique())
    ag_motifs = len(df.motifAGEpitope.unique())
    #median_sizeCDR3 = df.sizeCDR3.median()
    #mean_sizeCDR3 = df.sizeCDR3.mean()
    ag_stats[ag] = {'hot_spots':hot_spots,'ag_motifs':ag_motifs}

In [36]:
ag_ranked_dict = dict(sorted(ag_stats.items(), key=lambda x: x[1]['ag_motifs']))

In [39]:
ep_ranked_ag = list(ag_ranked_dict.keys())

In [40]:
ep_ranked_ag

['1FBI',
 '3VRL',
 '1OB1',
 '1WEJ',
 '1NSN',
 '3RAJ',
 '2YPV',
 '5E94',
 '1H0D',
 '1ADQ']

In [30]:
#join with performance plot
folder_2_task = {'1v9': 'vs 9', 'high_vs_95low': 'vs Non-binder', 'high_vs_looser': 'vs Weak', '1v1': 'vs 1'}
perf_df = pd.read_csv('../data/openset_performance.tsv', sep='\t')
perf_df[['train_task', 'train_ag', 'train_neg', 'seed_id', 'split_id']] = perf_df.task_1.str.split('__', expand=True)
perf_df[['test_task', 'test_ag', 'test_neg', 'test_seed_id', 'test_split_id']] = perf_df.task_2.str.split('__', expand=True)
perf_df.drop(['task_1', 'task_2'], inplace=True, axis=1)
task_rename = {'ONE_VS_NINE': 'vs 9', 'HIGH_VS_95LOW': 'vs Non-binder', 'HIGH_VS_LOOSER': 'vs Weak'}
perf_df['train_task'] = perf_df['train_task'].apply(lambda x: task_rename[x])
perf_df['test_task'] = perf_df['test_task'].apply(lambda x: task_rename[x])

df_closed = perf_df[perf_df['train_task'] == perf_df['test_task']].copy()

In [50]:
closed_perf_order = dict()
for task, df in df_closed.groupby('train_task'):
    order = list(df.groupby('train_ag').mean().sort_values(by='acc').index)
    closed_perf_order[task] = order

In [53]:
logit_energy_corr_path = "../data/07e_LogitEnergyCorrelations.tsv"
df_logits = pd.read_csv(logit_energy_corr_path, sep="\t")
df_logits_old = pd.read_csv("../data/07e_LogitEnergyCorrelations_old.tsv", sep="\t")
df_logits_new = pd.concat(
    [
        df_logits.query("task_type == '1v1'").copy(),
        df_logits_old.query("task_type != '1v1'").copy(),
    ],
    axis=0
)
df_logits_new.task_type.value_counts()
df_logits = df_logits_new.copy()

In [55]:
df_logits.head()

,task_type,ag_pos,ag_neg,seed_id,split_id,r,p,r2,r_pos,r_pos2,...,Energy_std,logits_pos_mean,logits_pos_std,Energy_pos_mean,Energy_pos_std,logits_neg_mean,logits_neg_std,Energy_neg_mean,Energy_neg_std,corr_type
0,1v1,3VRL,1NSN,0,42,-0.585052,0.000000e+00,0.342285,-0.585052,0.342285,...,1.678374,14.436833,6.624078,-98.309468,1.678374,NaN,NaN,NaN,NaN,NaN
1,1v1,3VRL,3RAJ,0,42,-0.334859,2.828910e-131,0.112130,-0.334859,0.112130,...,1.678374,8.870269,4.636723,-98.309468,1.678374,NaN,NaN,NaN,NaN,NaN
2,1v1,3VRL,5E94,0,42,-0.151120,6.265870e-27,0.022837,-0.151120,0.022837,...,1.678374,13.560961,6.124475,-98.309468,1.678374,NaN,NaN,NaN,NaN,NaN
3,1v1,3VRL,1H0D,0,42,-0.522465,0.000000e+00,0.272969,-0.522465,0.272969,...,1.678374,15.293151,7.849567,-98.309468,1.678374,NaN,NaN,NaN,NaN,NaN
4,1v1,3VRL,1WEJ,0,42,-0.537149,0.000000e+00,0.288529,-0.537149,0.288529,...,1.678374,13.008614,6.376433,-98.309468,1.678374,NaN,NaN,NaN,NaN,NaN


In [57]:
logit_energy_order = dict()
for task, df in df_logits.groupby('task_type'):
    order = list(df.groupby('ag_pos').mean().sort_values(by='r_pos').index)
    logit_energy_order[task] = order

In [58]:
logit_energy_order

{'1v1': ['3VRL',
  '5E94',
  '1FBI',
  '1OB1',
  '3RAJ',
  '1WEJ',
  '2YPV',
  '1ADQ',
  '1H0D',
  '1NSN'],
 '1v9': ['3VRL',
  '5E94',
  '1FBI',
  '1OB1',
  '3RAJ',
  '1WEJ',
  '2YPV',
  '1ADQ',
  '1H0D',
  '1NSN'],
 'high_vs_95low': ['3VRL',
  '1FBI',
  '5E94',
  '1WEJ',
  '1OB1',
  '3RAJ',
  '1NSN',
  '2YPV',
  '1H0D',
  '1ADQ'],
 'high_vs_looser': ['1FBI',
  '3VRL',
  '5E94',
  '1OB1',
  '3RAJ',
  '1H0D',
  '1NSN',
  '2YPV',
  '1WEJ',
  '1ADQ']}

In [59]:
from scipy.stats import kendalltau

In [60]:
ep_ranked_ag
closed_perf_order

['1FBI',
 '3VRL',
 '1OB1',
 '1WEJ',
 '1NSN',
 '3RAJ',
 '2YPV',
 '5E94',
 '1H0D',
 '1ADQ']

In [86]:
tau, p = kendalltau(ep_ranked_ag, logit_energy_order['high_vs_looser'])

In [87]:
tau

0.28888888888888886

In [81]:
logit_energy_order.keys()

dict_keys(['1v1', '1v9', 'high_vs_95low', 'high_vs_looser'])

In [76]:
closed_perf_order['vs 9']

['3RAJ',
 '1ADQ',
 '1WEJ',
 '1H0D',
 '2YPV',
 '5E94',
 '1OB1',
 '1NSN',
 '1FBI',
 '3VRL']

In [66]:
pd.DataFrame(ag_ranked_dict)

,1FBI,3VRL,1OB1,1WEJ,1NSN,3RAJ,2YPV,5E94,1H0D,1ADQ
hot_spots,2,1,1,3,1,4,3,3,3,4
ag_motifs,8,11,13,16,20,22,22,33,37,40


In [77]:
df_closed

,acc,recall,precision,f1,train_task,train_ag,train_neg,seed_id,split_id,test_task,test_ag,test_neg,test_seed_id,test_split_id
0,0.972986,0.9750,0.971116,0.973054,vs 9,3VRL,auto,0,42,vs 9,3VRL,auto,0,42
4,0.996600,0.9980,0.995213,0.996605,vs Non-binder,3VRL,auto,0,42,vs Non-binder,3VRL,auto,0,42
8,0.953400,0.9604,0.947140,0.953724,vs Weak,3VRL,auto,0,42,vs Weak,3VRL,auto,0,42
9,0.972686,0.9782,0.967557,0.972849,vs 9,3VRL,auto,1,42,vs 9,3VRL,auto,1,42
13,0.996200,0.9986,0.993830,0.996209,vs Non-binder,3VRL,auto,1,42,vs Non-binder,3VRL,auto,1,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.988900,0.9954,0.982626,0.988972,vs Non-binder,1OB1,auto,0,3,vs Non-binder,1OB1,auto,0,3
800,0.918700,0.9092,0.926809,0.917920,vs Weak,1OB1,auto,0,3,vs Weak,1OB1,auto,0,3
801,0.944472,0.9350,0.953109,0.943968,vs 9,1OB1,auto,0,4,vs 9,1OB1,auto,0,4
805,0.988000,0.9946,0.981642,0.988079,vs Non-binder,1OB1,auto,0,4,vs Non-binder,1OB1,auto,0,4


In [12]:
ag_epitopes = dict()
for ag in config.ANTIGENS:
    ag_path = glob.glob(f'../data/Slide_importance/{ag}_*_Task4_Slice_ParaEpi.txt')[0]
    df = pd.read_csv(ag_path, sep='\t')
    ag_motifs = df.agregatesAGEpitope.unique()
    #median_sizeCDR3 = df.sizeCDR3.median()
    #mean_sizeCDR3 = df.sizeCDR3.mean()
    ag_epitopes[ag] = list(ag_motifs)

In [5]:
import re
from Bio import AlignIO
from Bio.Align.Applications import ClustalOmegaCommandline

In [13]:
for ag in config.ANTIGENS:
    with open(f"../data/epitopes_info/{ag}_epitope_seq.fasta", "w") as fasta_file:
        seq_4_alig = list(map(lambda x:re.sub('\d', '', x), ag_epitopes[ag]))
        for i, seq in enumerate(seq_4_alig):
            fasta_file.write(f'>{i}\n{seq}\n')


In [4]:
df_ex.head()

,Slide,Label,hotspot_ID,seqAGEpitope,motifAGEpitope,agregatesAGEpitope,chemicalAGEpitope,seqABParatope,motifABParatope,agregatesABParatope,chemicalABParatope,AAcompoFullSlice,sizeCDR3
0,LLGLYWYFDVW,1ADQ_A,1ADQ_A_H1,S1W1V1D1V2Q1V3N1A1K1T2V1L2H1N1Y1,X1--X1--X1X1--X2X1X3--X1X1X1--X2X1X2X1--X1--X1,S1--W1--V1D1--V2Q1V3--N1A1K1--T2V1L2H1--N1--Y1,p1--r1--n1c1--n2p1n3--p1n1c1--p2n1n2c1--p1--r1,L3L4G3L1Y2Y2F2V2W2*,X3X4X3X1X2--X2X2--X2X2*,L3L4G3L1Y2--Y2F2--V2W2*,n3n4n3n1r2--r2r2--n2r2*,0_0_0.0909091_0_0.0909091_0.0909091_0_0_0_0.27...,15
1,YYSNYWYFDVW,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,Y2Y3N2Y1W3Y1F4D1V4W3*,X2X3--X2X1X3X1X4X1X4X3*,Y2Y3--N2Y1W3Y1F4D1V4W3*,r2r3--p2r1r3r1r4c1n4r3*,0_0_0.0909091_0_0.0909091_0_0_0_0_0_0_0.090909...,14
2,AILLLRGKAYW,1ADQ_A,1ADQ_A_H1,L1R1P1W1V1D2V1Q1V1A1K1T1V1L1H1Q3N1W1L1Y1,X1--X1--X1--X1--X1X2--X1X1X1--X1X1--X1X1X1X1X3...,L1--R1--P1--W1--V1D2--V1Q1V1--A1K1--T1V1L1H1Q3...,n1--c1--n1--r1--n1c2--n1p1n1--n1c1--p1n1n1c1p3...,A2I3L4L3L1R1G1K1A2Y1W4*,X2X3X4X3X1X1X1X1X2X1X4*,A2I3L4L3L1R1G1K1A2Y1W4*,n2n3n4n3n1c1n1c1n2r1r4*,0.181818_0_0_0_0_0.0909091_0_0.0909091_0.09090...,12
3,YYDYDWYFDVW,1ADQ_A,1ADQ_A_H2,P1V1F1V3D3V1S1Q2Q1V1T1K1P1E1Y1R2V1V1,X1--X1X1--X3--X3X1X1X2--X1X1--X1X1X1--X1--X1X2...,P1--V1F1--V3--D3V1S1Q2--Q1V1--T1K1P1--E1--Y1R2...,n1--n1r1--n3--c3n1p1p2--p1n1--p1c1n1--c1--r1c2...,Y2Y3Y2D1W3Y1F4D1V4W3*,X2X3--X2X1X3X1X4X1X4X3*,Y2Y3--Y2D1W3Y1F4D1V4W3*,r2r3--r2c1r3r1r4c1n4r3*,0_0_0.272727_0_0.0909091_0_0_0_0_0_0_0_0_0_0_0...,15
4,SLIYYGYGAWF,1ADQ_A,1ADQ_A_H1,L1R1P1W1V1D2V1Q1V1A1K1T1V1L1H1Q3N1W1L1Y1,X1--X1--X1--X1--X1X2--X1X1X1--X1X1--X1X1X1X1X3...,L1--R1--P1--W1--V1D2--V1Q1V1--A1K1--T1V1L1H1Q3...,n1--c1--n1--r1--n1c2--n1p1n1--n1c1--p1n1n1c1p3...,S2L3I4Y3Y1G1Y1G1A2W1F4*,X2X3X4X3X1X1X1X1X2X1X4*,S2L3I4Y3Y1G1Y1G1A2W1F4*,p2n3n4r3r1n1r1n1n2r1r4*,0.0909091_0_0_0_0.0909091_0.181818_0_0.0909091...,16


In [14]:
for i in ag_epitopes['5E94']:
    print(i)

T1--D1E1--H1V1Y1--F2--L1Q1--S2S2L1P1W1R2D1L2--L1
D1E1--V1Y1--F1--L1Q1--S2S2--W1R1D1L2--L2--L1Y1
Q1--S1--A1--W1--F1--N2--S2--P1W1--P1Q1--H2V1Y1R1F3--L1
A1--W1--F1--N2--S2--P1--P1Q1--H2V1Y1R1F3--L1Q1--S1--W1
E1--H1V1Y1--F2--L1Q1--S2S1--W1--D1L2--L1L2--L1Y1
D1E1--Y1--F1--L1Q1--S2S2--W1R1D1L2S1--L3--L1Y1
E1--G1H1V1Y1--F2--L1Q1--S2S1--W1--D1L2--L2--L1Y1
E1--V1Y1--F1--L1Q1--S2S1--W1--D1L2S1--L3--L1Y1
W1--R1Q1--R1S1--A1--W1--P1--F2--N2--S2--P1W2--P1Q1--H1--R1F1C1T1
T1--K1W1--R1Q1--R1S1L1--P1--A1--W2P1--G1--V1N1--S1--P2W3--P1--A1
P1--F2--N1--Q1--H2V1Y1R1F3C1T1--L1Q1--S1--W1--L1
E1--V1Y1--F1--L1Q1--S2S1--W1--D1L2--Q1L1L2--L2Y1
Q1--S1--A1--W1--N1--S2--P1W1--P1Q1--H2V1Y1--F2--L1Q1--S1--W1--L1
E1--F1--N1--H1V1Y1R1F3--L1Q1--S2S1--W1--D1L2--L1
E1--F1--N1--Q1--H2V1Y1R1F2--L1Q1--S2S1--W1--D1L2--L1
Q1--S2L1--P1--A1--W2P1--G1--F1V1N3--S2--P1W1--P1Q1--H1--R1
D1E1--H1V1Y1--F2--L1Q1--S2S2--W1R1D1L2S1--L2
D1E1--G1H1V1Y1--F2--L1Q1--S2S2--W1R1D1L2--E1--L1
C1--T1--D1E1--V1Y1--F1--L1Q1--S2S2L1P1W1R2D1L2--L1
E1-

In [16]:
task_template = {'ONE_VS_NINE': '{ag}__vs__9', 'HIGH_VS_95LOW': '{ag}_high__vs__{ag}_95low', 'HIGH_VS_LOOSER': '{ag}_high__vs__{ag}_looser'}
tasks_dict = {'ONE_VS_NINE': '1v9', 'HIGH_VS_95LOW': 'high_vs_95low', 'HIGH_VS_LOOSER': 'high_vs_looser'}

In [66]:
#mps_device = torch.device("mps")
for ag in config.ANTIGENS:
    print(ag)

    #convert aa_degrees column to list
    task  = 'HIGH_VS_LOOSER'
    seed_id, split_id = 0,42
    model_path = f"./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed_id}/split_{split_id}/{task_template[task].format(ag=ag)}/swa_model/data/model.pth"
    data_path = glob.glob(f"./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed_id}/split_{split_id}/{task_template[task].format(ag=ag)}/*test_dataset.tsv")[0]
    df_test = pd.read_csv(data_path, sep='\t')
    df_test['X'] = df_test['Slide'].apply(lambda x: [[onehot_encode(x)]])
    X = torch.tensor(df_test['X'].tolist(), dtype=torch.float32).to('cpu')
    model = torch.load(model_path, map_location=torch.device('cpu'))
    y_pred = model(X, return_logits = False)
    df_test['y_pred'] = y_pred.cpu().detach().numpy().reshape(-1).round()
    epi_path = glob.glob(f'../data/Slide_importance/{ag}_*_Task4_Slice_ParaEpi.txt')[0]
    df_epitope = pd.read_csv(epi_path, sep='\t')
    df_test = df_test.merge(df_epitope, on='Slide', how='left')
    ep_tpos = dict(df_test[(df_test['y']== 1) & (df_test['y_pred']==1)].agregatesAGEpitope.value_counts())
    ep_fneg = dict(df_test[(df_test['y']== 1) & (df_test['y_pred']==0)].agregatesAGEpitope.value_counts())
    ep_tpos_fneg = dict()
    for ep, count in ep_fneg.items():
        if ep in  ep_tpos:
             ep_tpos_fneg[ep] = str(ep_tpos[ep] -  ep_fneg[ep])
        else:
            ep_tpos_fneg[ep] = str(ep_fneg[ep]*(-1))
    with open(f'../data/epitopes_info/{ag}_ep_diff.json', 'w') as f:
        json.dump(ep_tpos_fneg, f)


3VRL
1NSN
3RAJ
5E94
1H0D
1WEJ
1ADQ
1FBI
2YPV
1OB1


In [48]:
dict(df_test[(df_test['y']== 1) & (df_test['y_pred']==1)].agregatesAGEpitope.value_counts())

{'S1--N1S1G1--L3--V1--N2--K1--I2--C2--C1--P1--F2D2': 3362,
 'S1--Q1C1--Q1--N1S1G2--L3--V1--N1--I1--C2--C1--P1--F2D2': 1180,
 'N1--S2--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 73,
 'N1--S2--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 22,
 'S1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 6,
 'N1--S2--Q1--S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 5,
 'S1--N1S1G1--L3--V1--N1--I1--C2--C1T1--P1--F2D2': 3}

In [55]:
ep_fneg = df_test[(df_test['y']== 1) & (df_test['y_pred']==0)].agregatesAGEpitope.value_counts().to_dict()

In [57]:
ep_fneg.items()

dict_items([('S1--N1S1G1--L3--V1--N2--K1--I2--C2--C1--P1--F2D2', 225), ('S1--Q1C1--Q1--N1S1G2--L3--V1--N1--I1--C2--C1--P1--F2D2', 87), ('N1--S2--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2', 11), ('S1--N1S1G1--L3--V1--N1--I1--C2--C1T1--P1--F2D2', 8), ('N1--S2--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2', 8), ('S1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2', 5), ('N1I1--H1Q2C1V3--Q1--S1G2--R1--L1--E1--L2--I1--C1--F1D1', 2), ('N1I1--H1Q2--V2--N1S1G1--R1--L1--E1--L3--V1--N1--I1--C1--F1D1', 2), ('S1--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2', 1)])

In [ ]:
S1--N1S1G1--L3--V1--N2--K1--I2--C2--C1--P1--F2D2          3362
S1--Q1C1--Q1--N1S1G2--L3--V1--N1--I1--C2--C1--P1--F2D2    1180
N1--S2--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2            73
N1--S2--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2        22
S1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2                 6
N1--S2--Q1--S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2           5
S1--N1S1G1--L3--V1--N1--I1--C2--C1T1--P1--F2D2               3

S1--N1S1G1--L3--V1--N2--K1--I2--C2--C1--P1--F2D2                225
S1--Q1C1--Q1--N1S1G2--L3--V1--N1--I1--C2--C1--P1--F2D2           87
N1--S2--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2             11
S1--N1S1G1--L3--V1--N1--I1--C2--C1T1--P1--F2D2                    8
N1--S2--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2                  8
S1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2                      5
N1I1--H1Q2C1V3--Q1--S1G2--R1--L1--E1--L2--I1--C1--F1D1            2
N1I1--H1Q2--V2--N1S1G1--R1--L1--E1--L3--V1--N1--I1--C1--F1D1      2
S1--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2                  1

In [50]:
dict(df_test[(df_test['y']== 1) & (df_test['y_pred']==0)].agregatesAGEpitope.value_counts())

{'S1--N1S1G1--L3--V1--N2--K1--I2--C2--C1--P1--F2D2': 225,
 'S1--Q1C1--Q1--N1S1G2--L3--V1--N1--I1--C2--C1--P1--F2D2': 87,
 'N1--S2--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 11,
 'S1--N1S1G1--L3--V1--N1--I1--C2--C1T1--P1--F2D2': 8,
 'N1--S2--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 8,
 'S1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 5,
 'N1I1--H1Q2C1V3--Q1--S1G2--R1--L1--E1--L2--I1--C1--F1D1': 2,
 'N1I1--H1Q2--V2--N1S1G1--R1--L1--E1--L3--V1--N1--I1--C1--F1D1': 2,
 'S1--Q1--N1S1G1--L3--V1--N1--I1--C2--C1--P1--F2D2': 1}